# Benchmark Performance of argmin()

In [2]:
using BenchmarkTools

In [3]:
x = rand(450);

In [25]:
@btime findmin(x)

  516.057 ns (1 allocation: 32 bytes)


(0.0017632793527515567, 315)

In [26]:
@btime findmin(@view x[begin:end])

  567.935 ns (3 allocations: 112 bytes)


(0.0017632793527515567, 315)

In [27]:
@btime findmin(@view x[1:449])

  537.698 ns (2 allocations: 80 bytes)


(0.0017632793527515567, 315)

In [28]:
@btime findmin(@view x[1:448])

  536.376 ns (2 allocations: 80 bytes)


(0.0017632793527515567, 315)

In [29]:
@benchmark findmin(x)

BenchmarkTools.Trial: 10000 samples with 192 evaluations.
 Range (min … max):  516.714 ns …  8.406 μs  ┊ GC (min … max): 0.00% … 92.77%
 Time  (median):     522.135 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   537.012 ns ± 83.419 ns  ┊ GC (mean ± σ):  0.15% ±  0.93%

  ██▆▅▄▂   ▁▄▄▄▄▃▃▂▁▁ ▁▃▄▄▃▂▃▂▁▁ ▁▁  ▁▁▁▁                      ▂
  ██████▇▆▆████████████████████████████████▇▇▇▆▇▇▇▇▇▆▆▆▅▆▆▄▆▆▆ █
  517 ns        Histogram: log(frequency) by time       626 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.

In [30]:
@code_lowered findmin(x)

CodeInfo(
1 ─ %1 = Base.:(var"#findmin#860")(Base.:(:), #self#, A)
└──      return %1
)

In [9]:
using LoopVectorization

In [31]:
fast_findmin(dij, n) = begin
    best = 1
    @inbounds dij_min = dij[1]
    @turbo for here in 2:n
        newmin = dij[here] < dij_min
        best = newmin ? here : best
        dij_min = newmin ? dij[here] : dij_min
    end
    dij_min, best
end

fast_findmin (generic function with 1 method)

In [32]:
@btime fast_findmin(x, 450)

  97.736 ns (1 allocation: 32 bytes)


(0.0017632793527515567, 315)

In [33]:
@benchmark fast_findmin(x, 450)

BenchmarkTools.Trial: 10000 samples with 946 evaluations.
 Range (min … max):   97.736 ns …  1.478 μs  ┊ GC (min … max): 0.00% … 92.26%
 Time  (median):     100.598 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   102.964 ns ± 32.326 ns  ┊ GC (mean ± σ):  0.75% ±  2.24%

  ██▆▁ ▁▇█▆▃▅▆▅▃▃▃▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▂ ▂▁▁▁▁ ▁                     ▃
  ██████████████████████████████████████████████▇█▇▇▇▇▇▇▆▆▇▇▆▇ █
  97.7 ns       Histogram: log(frequency) by time       120 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.

In [34]:
@benchmark for j in 450:-1:1 fast_findmin(x, j) end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  26.250 μs …  1.311 ms  ┊ GC (min … max): 0.00% … 96.40%
 Time  (median):     26.875 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   28.079 μs ± 20.770 μs  ┊ GC (mean ± σ):  1.25% ±  1.67%

  ▃▇█▆▆▆▅▅▂▁▂ ▁ ▁▁   ▁▂▂▂▂▂▁  ▁                               ▂
  ███████████████████████████████▇▇▇▆▆▆▅▆▆▆▆▇▇▆▇▆▄▅▅▆▅▅▆▅▄▅▄▅ █
  26.2 μs      Histogram: log(frequency) by time        37 μs <

 Memory estimate: 14.06 KiB, allocs estimate: 450.

In [35]:
@benchmark for j in 450:-1:1 findmin(@view x[1:j]) end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  125.542 μs …  1.908 ms  ┊ GC (min … max): 0.00% … 92.30%
 Time  (median):     127.250 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   131.795 μs ± 56.608 μs  ┊ GC (mean ± σ):  1.40% ±  3.04%

  ▃▆██▆▃▃▄▄▃▃▄▄▄▃▃▃▃▃▂▂▂▂▃▂▂▂▂▂▁▁▂▁▁▁▁▁▁ ▁                     ▂
  █████████████████████████████████████████▇▇▇▇█▇██▇▇▆▆▆▆▇▇▅▅▇ █
  126 μs        Histogram: log(frequency) by time       148 μs <

 Memory estimate: 49.22 KiB, allocs estimate: 1350.